## [**Training Notebook**](https://colab.research.google.com/drive/1o5KGZO4s4d3wTIqEUHbGzbXiCmdLniTx?usp=sharing)

### **Library Imports**

In [1]:
import os
import re
import cv2
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch import nn, optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader as DL
from torchvision import models, transforms

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

### **Utilites and Constants**

In [2]:
SEED = 42
SIZE = 256
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
TRANSFORM = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize([0.37250, 0.37591, 0.37537],
                                                     [0.20316, 0.21171, 0.21160]),
                                ])
MODEL_PATH = "../input/fgvc9-r50-full-na256-oclr/resnet50-state.pt"

le = LabelEncoder()

def breaker(num: int=50, char: str="*") -> None:
    print("\n" + num*char + "\n")

In [3]:
def get_image(path: str, size: int) -> np.ndarray:
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2RGB)
    return cv2.resize(src=image, dsize=(size, size), interpolation=cv2.INTER_AREA)

### **Get Label Encoding Transform**

In [4]:
train_df = pd.read_csv("../input/sorghum-id-fgvc-9/train_cultivar_mapping.csv")
train_df = train_df.drop(index=train_df.index[train_df.image == ".DS_Store"])

labels    = train_df.cultivar.copy().values 
labels = le.fit_transform(labels)

### **Configuration**

In [5]:
class CFG(object):
    def __init__(self, 
                 seed: int = 42,
                 n_splits: int = 5,
                 batch_size: int = 16,
                 epochs: int = 25,
                 early_stopping: int = 5,
                 lr: float = 1e-4,
                 wd: float = 0.0,
                 max_lr: float = 1e-3,
                 pct_start: float = 0.2,
                 steps_per_epoch: int = 100,
                 div_factor: int = 1e3, 
                 final_div_factor: float = 1e3,
                 ):
        
        self.seed = seed
        self.n_splits = n_splits
        self.batch_size = batch_size
        self.epochs = epochs
        self.early_stopping = early_stopping
        self.lr = lr
        self.wd = wd
        self.max_lr = max_lr
        self.pct_start = pct_start
        self.steps_per_epoch = steps_per_epoch
        self.div_factor = div_factor
        self.final_div_factor = final_div_factor
    
cfg = CFG(seed=SEED)

### **Dataset Template**

In [6]:
class DS(Dataset):
    def __init__(self, base_path: str, filenames: np.ndarray, transform):
        self.base_path = base_path
        self.filenames = filenames
        self.transform = transform
    
    def __len__(self):
        return self.filenames.shape[0]
    
    def __getitem__(self, idx):
        return self.transform(get_image(os.path.join(self.base_path, self.filenames[idx]), SIZE))

### **Model**

In [7]:
class MyResnet(nn.Module):
    def __init__(self):
        super(MyResnet, self).__init__()

        self.model = models.resnet50(pretrained=False, progress=True)
        self.model.fc = nn.Linear(in_features=self.model.fc.in_features, out_features=100)

    def forward(self, x):
        return nn.LogSoftmax(dim=1)(self.model(x))

### **Predict Helper**

In [8]:
def predict(model=None, dataloader=None, path=None) -> np.ndarray:
    model.load_state_dict(torch.load(path, map_location=DEVICE)["model_state_dict"])
    model.to(DEVICE)    
    model.eval()
    
    y_pred = torch.zeros(1, 1).to(DEVICE)
    
    for X in dataloader:
        X = X.to(DEVICE)
        with torch.no_grad():
            output = torch.argmax(torch.exp(model(X)), dim=1)
        y_pred = torch.cat((y_pred, output.view(-1, 1)), dim=0)
    
    return y_pred[1:].detach().cpu().numpy()

### **Submission**

In [9]:
ss_df = pd.read_csv("../input/sorghum-id-fgvc-9/sample_submission.csv")

ts_data_setup = DS("../input/sorghum-id-fgvc-9/test", ss_df.filename.copy().values, TRANSFORM)
ts_data = DL(ts_data_setup, batch_size=cfg.batch_size, shuffle=False)

y_pred = predict(model=MyResnet().to(DEVICE), dataloader=ts_data, path=MODEL_PATH)
y_pred = le.inverse_transform(y_pred.astype("uint8"))

ss_df["cultivar"] = y_pred
ss_df.to_csv("submission.csv", index=False)

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:154: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
